In [1]:
# systems
import os

# scraping websites
import json
import requests
from bs4 import BeautifulSoup

# language processing
from sutime import SUTime

## Get listings

Set up session

In [2]:
with open('../credentials/MIT_login.json') as f: 
    login_data = json.load(f)

In [3]:
s = requests.session()
s.post("http://mailman.mit.edu/mailman/private/mitml/", login_data)

<Response [200]>

In [7]:
def get_soup(url, session=s): 
    response = session.post(url)
    return BeautifulSoup(response.text, 'html.parser')

Get listings by batch

In [5]:
list_id = "mitml"
base_url = "http://mailman.mit.edu/mailman/private/"

In [8]:
# Request home 
home_url = os.path.join(base_url, list_id)
home_soup = get_soup(home_url)

# Get batch urls (by date)
batch_urls = [os.path.join(home_url, subpage.get('href')) for subpage in home_soup.body.table.find_all(href=True) if 'date' in subpage.get('href')]

In [17]:
listings = {}

for batch_url in batch_urls:
    # For each batch, find indices to each listing
    batch_soup = get_soup(batch_url)

    indices = [subpage.get('href') for subpage in batch_soup.body.find_all('ul')[1].find_all(href=True)]
    
    # Generate full paths and append to list
    for index in indices[::-1]: 
        url = os.path.join(os.path.dirname(batch_url), index)
        listings[index] = {
            "url" : url, 
            "html": get_soup(url)
        }
        
print("{} listings found.".format(len(listing_urls)))

481 listings found.


Write file locally

In [50]:
local_dir = '../listings/'
urls = []

for index, dic in listings.items(): 
    
    urls.append(dic['url'])
    
    with open(os.path.join(local_dir, list_id, index), 'w') as f: 
        f.write(str(dic['html']))
        
with open(os.path.join(local_dir, list_id, 'urls.txt'), 'w') as f: 
    f.write('\n'.join(urls))